In [7]:
#from __future__ import print_function, division, absolute_import, unicode_literals

import numpy as np
import random
import math
import pickle
import os
#import matlab.engine

#eng = matlab.engine.start_matlab()
#eng.triarea(nargout=0)

sigma = 10

plateau_min = -2
plateau_max = 2

r_min = 1
r_max = 200

#def generate_voronoi_diagram(width, height, cnt=50, bin_vector= [0, 30, 60]):
width = 512 
height = 512
cnt=50
bin_vector= [0, 30, 60]

num_cells = cnt

# Modified these paths
script_dir = os.path.dirname(os.getcwd()) #<-- absolute dir the script is in
rel_path = 'tf_unet/tf_unet/Attenuation.pkl'
abs_file_path = os.path.join(script_dir, rel_path)

# This is the part that imports the files
with open(abs_file_path,'rb') as f:  # Python 3: open(..., 'rb')
    Attenuation, Attenuation2, Spec = pickle.load(f)
Ev = Spec[:,0]
print(Ev)
Int = Spec[:,1]

#Convert the bin vector from energies to indices
nbins = len(bin_vector)-1

for ii in range(nbins+1):
    bin_vector[ii] = min(range(len(Ev)), key=lambda i: abs(Ev[i]-bin_vector[ii]))

print(bin_vector)

# Initialize the variables
image = np.ones((width,height,nbins))
labels = np.zeros([width,height],dtype=np.bool)
imgx, imgy = width, height
nx = [] # hold the x value
ny = [] # holds the y value
nr = np.zeros([num_cells,len(bin_vector)-1]) # low energy image
label = [] # labels whether or not the cell has cartilage
total_intensities = []

# finding the fractions of the fluence
for ii in range(len(bin_vector)-1):
    total_intensities.append(sum(Int[bin_vector[ii]:bin_vector[ii+1]]))

pixel_value = []


for i in range(num_cells):

    # choosing random place for the cell
    nx.append(random.randrange(imgx))
    ny.append(random.randrange(imgy))
    rr = random.random()
    pp = random.random()

    # making a random value for the cartilage
    if i%4==0:
        t_cart = random.uniform(0.2,3.9)
        label.append(0)
    else:
        label.append(1)
        t_cart = 0.

    # Making a random value for the muscle
    t_muscle = 6 - t_cart

    # Attenuating
    pixel_value = 0.
    for jj in range(nbins): 
    # finding the attenuation for the low image
        for ii,energy in enumerate(Ev[bin_vector[ii]:bin_vector[ii+1]]):

            mu = np.interp(energy/1000.,Attenuation[:,0],Attenuation[:,7])
            mu2 = np.interp(energy/1000.,Attenuation2[:,0],Attenuation2[:,7])

            pixel_value += Int[ii+30]*np.exp(-mu*t_muscle*1.05)*np.exp(-mu2*t_cart*1.1)

        nr[i,jj] = pixel_value

        pixel_value = 0.


for y in range(imgy):
    for x in range(imgx):
        dmin = math.hypot(imgx-1, imgy-1)
        j = -1
        for i in range(num_cells):
            d = math.hypot(nx[i]-x, ny[i]-y)
            if d < dmin:
                dmin = d
                j = i

        # Generating some poisson noise
        for jj in range(nbins): 
            image[x,y,jj] = nr[j,jj]
            #image[x,y,jj] = random.gauss(nr[j,jj],(nr[j,jj])**(0.5))#/(total_intensity_low)
            #image2[x,y,0] = random.gauss(nr2[j],(nr2[j])**(0.5))#/(total_intensity_high)

            #image[x,y,jj] = image[x,y,0]/image2[x,y,0]
        labels[x,y] = label[j]

#image -= np.amin(image)
#image /= np.amax(image)

#return image, labels

[ 6.5  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.
 13.5 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.
 20.5 21.  21.5 22.  22.5 23.  23.5 24.  24.5 25.  25.5 26.  26.5 27.
 27.5 28.  28.5 29.  29.5 30.  30.5 31.  31.5 32.  32.5 33.  33.5 34.
 34.5 35.  35.5 36.  36.5 37.  37.5 38.  38.5 39.  39.5 40.  40.5 41.
 41.5 42.  42.5 43.  43.5 44.  44.5 45.  45.5 46.  46.5 47.  47.5 48.
 48.5 49.  49.5 50.  50.5 51.  51.5 52.  52.5 53.  53.5 54.  54.5 55.
 55.5 56.  56.5 57.  57.5 58.  58.5 59.  59.5 60. ]
[0, 47, 107]


IndexError: list index out of range

In [4]:
def get_image_gen(nx, ny, **kwargs):
    def create_batch(n_image):
        
        X = np.zeros((n_image,nx,ny,1))
        Y = np.zeros((n_image,nx,ny,2))
        
        #for i in range(n_image):
        #X[i],Y[i,:,:,1] = create_image_and_label(nx,ny, **kwargs)
        X[i],Y[i,:,:,:] = generate_voronoi_diagram(nx,ny, **kwargs)
        #Y[i,:,:,0] = 1-Y[i,:,:,1]
            
        return X,Y
    
    create_batch.channels = 1
    create_batch.n_class = 2
    return create_batch

In [5]:
nx = 512
ny = 512

generator = get_image_gen(nx, ny, cnt=80)

x_test, y_test = generator(4)

%matplotlib
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,2, sharey=True, figsize=(8,4))
ax[0].imshow(x_test[0,...,0], aspect="auto")
ax[1].imshow(y_test[0,...,1], aspect="auto")